In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - prepare_zenodo.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Get paths to download the final data products, and collect in simpler
directories for Zenodo upload.
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, copy
import shutil
import numpy as np

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Notebook setup
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
# feh_range_fit = copy.deepcopy( # Need to choose here


## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max],ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid
# apogee_SF,apogee_effSF_grid_inclArea,apogee_effSF_grid_inclArea_Jac = other_stuff

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.9]
# dfs = putil.load_distribution_functions(df_filename, betas)

# ## Load the APOGEE field information, can also similarly load 
# ## '...apogee_field_glons.npy', '...apogee_field_glats.npy', 
# ## '...apogee_field_location_ids.npy'
# apogee_fields = np.load(ga_dir+'apogee_fields.npy')

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             # 'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

### GS/E data

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk,
             ]

selec = ['eLz','AD','JRLz',None]
fit_type = ['gse','gse','gse','all']
_version = '100w_1e4n'
versions = [_version]*4
feh_range_fit = [feh_range_gse,feh_range_gse,feh_range_gse,feh_range_all]
verbose = False

In [ ]:
zenodo_dir = os.path.join(fit_dir,'zenodo/')
os.makedirs(zenodo_dir, exist_ok=True)
zenodo_gse_dir = os.path.join(zenodo_dir,'gse/')
os.makedirs(zenodo_gse_dir, exist_ok=True)
zenodo_all_dir = os.path.join(zenodo_dir,'all/')
os.makedirs(zenodo_all_dir, exist_ok=True)

for d1 in ['eLz','AD','JRLz']:
    os.makedirs(os.path.join(zenodo_gse_dir,d1), exist_ok=True)
    for d2 in densfuncs:
        os.makedirs(os.path.join(zenodo_gse_dir,d1,d2.__name__), exist_ok=True)

for d in densfuncs:
    os.makedirs(os.path.join(zenodo_all_dir,d.__name__), exist_ok=True)

In [ ]:
# Loop over the fit type / selection / version the loop over density profile
for i in range(4):
    for j in range(8):
        hf = pmass.HaloFit(densfunc=densfuncs[j], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit[i], **hf_kwargs)
        samples_filename = os.path.join(hf.fit_data_dir,'samples.npy')
        if fit_type[i] == 'gse':
            samples_dest_filename = os.path.join(zenodo_gse_dir,selec[i],
                densfuncs[j].__name__,'samples.npy')
        elif fit_type[i] == 'all':
            samples_dest_filename = os.path.join(zenodo_all_dir,
                densfuncs[j].__name__,'samples.npy')

        print('\n')
        print('HF fit_data_dir is: ',hf.fit_data_dir)
        print('samples_filename is: ',samples_filename)
        print('samples_dest_filename is: ',samples_dest_filename)
        _data = np.load(samples_filename)
        print('samples shape is: ',_data.shape)

        shutil.copy(samples_filename,samples_dest_filename)